# People Analytics Case Study

## Libraries

In [1]:
# import libraries
import pymysql
from sqlalchemy import create_engine
import getpass

## Exporting Data for Tableau Visualization

In [3]:
# Connection parameters
bd = "hr_data"
password = "password"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd

# Create the connection engine
engine = create_engine(connection_string)

In [ ]:
# SQL query to execute
consulta = "SELECT * FROM hr_data.employees"

# Read the SQL query into a DataFrame
df_sql = pd.read_sql(consulta, con=engine)

# Display the first rows of the DataFrame
df_sql